In [1]:
%load_ext autoreload
import pandas as pd
import numpy as np

In [41]:
%autoreload
from src.xes.load_xes import load_raw_xes
from src.xes.xes_file import XesFile
from src.xes.print_xes_info import PrintXesInfo

In [4]:
xes = XesFile.read("data/BPI_Challenge_2013_closed_problems.xes")

In [42]:
printer = PrintXesInfo(xes)
printer.print_info()

Traces: 1487, events: 6660
Events: Average: 4.478816408876933, Min: 1, Max: 35

11 unique attributes in events:
            Attribute  Count Presence
            org:group   6659   100.0%
     resource country   6659   100.0%
 organization country   6659   100.0%
         org:resource   6659   100.0%
organization involved   6659   100.0%
             org:role   4581    68.8%
         concept:name   6659   100.0%
               impact   6659   100.0%
              product   6659   100.0%
 lifecycle:transition   6659   100.0%
       time:timestamp   6659   100.0%

"time:timestamp": 100.0% presence
average:      2011-11-01 09:52:43.071171328+00:00
min:          2006-01-11 14:49:42+00:00
max:          2012-05-31 21:49:06+00:00
average span: 178 days 21:14:13.334229992

"concept:name": 100.0% presence, 4 different values, top 4:
Accepted (4207), Completed (1568), Queued (875), Unmatched (10)

"lifecycle:transition": 100.0% presence, 7 different values, top 5:
In Progress (3066), Closed (156

In [38]:
printer.event_attributes_counts["Attribute"]

0                 org:group
1          resource country
2      organization country
3              org:resource
4     organization involved
5                  org:role
6              concept:name
7                    impact
8                   product
9      lifecycle:transition
10           time:timestamp
Name: Attribute, dtype: object

In [21]:
pd.to_datetime(xes.df["time:timestamp"], utc=True).mean()

Timestamp('2011-11-01 09:52:43.071171328+0000', tz='UTC')

In [24]:
xes.df.dtypes

org:group                object
resource country         object
organization country     object
org:resource             object
organization involved    object
org:role                 object
concept:name             object
impact                   object
product                  object
lifecycle:transition     object
time:timestamp           object
dtype: object

In [53]:
xes.traces[0].events_raw[0]["time:timestamp"]

datetime.datetime(2006, 1, 11, 15, 49, 42, tzinfo=tzoffset(None, 3600))

In [30]:
xes.traces[0].events.dtypes

org:group                                              object
resource country                                       object
organization country                                   object
org:resource                                           object
organization involved                                  object
org:role                                               object
concept:name                                           object
impact                                                 object
product                                                object
lifecycle:transition                                   object
time:timestamp           datetime64[ns, tzoffset(None, 3600)]
dtype: object

In [39]:
events_types = [event["time:timestamp"].__class__ for trace in xes.traces for event in trace.events_raw]

In [41]:
from collections import Counter
print(Counter(events_types))

Counter({<class 'datetime.datetime'>: 6660})


In [42]:
events_types = [trace.df["time:timestamp"].dtype for trace in xes.traces]

In [43]:
print(Counter(events_types))

Counter({dtype('O'): 705, datetime64[ns, tzoffset(None, 3600)]: 501, datetime64[ns, tzoffset(None, 7200)]: 281})


In [54]:
from datetime import timezone
print(xes.traces[0].events_raw[0]["time:timestamp"])
print(xes.traces[0].events_raw[0]["time:timestamp"].astimezone(timezone.utc))

2006-01-11 15:49:42+01:00
2006-01-11 14:49:42+00:00


In [63]:
min_per_trace = np.array([trace.events["time:timestamp"].min() for trace in xes.traces])
max_per_trace = np.array([trace.events["time:timestamp"].max() for trace in xes.traces])

In [64]:
min_per_trace

array([Timestamp('2006-01-11 14:49:42+0000', tz='UTC'),
       Timestamp('2006-11-07 09:00:36+0000', tz='UTC'),
       Timestamp('2007-03-20 08:06:25+0000', tz='UTC'), ...,
       Timestamp('2012-05-30 12:55:35+0000', tz='UTC'),
       Timestamp('2012-05-31 09:58:45+0000', tz='UTC'),
       Timestamp('2012-05-31 18:07:59+0000', tz='UTC')], dtype=object)

In [72]:
(max_per_trace - min_per_trace).astype('int64').mean()

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Timedelta'

In [77]:
min_per_trace[0].__class__

pandas._libs.tslibs.timestamps.Timestamp

In [83]:
pd.Series(max_per_trace - min_per_trace).mean()

Timedelta('178 days 21:14:13.334229992')

In [90]:
pd.Series(max_per_trace - min_per_trace).dt.days

0       2254
1       1900
2       1764
3       1713
4       1622
        ... 
1482       0
1483       0
1484       0
1485       0
1486       0
Length: 1487, dtype: int64

In [92]:
import matplotlib.pyplot as plt
# plt.hist(pd.Series(max_per_trace - min_per_trace).dt.days)

In [113]:
top = pd.Series({"a": 1, "b": 2}).nlargest(5)
print(top.to_string())

b    2
a    1


In [109]:
[f"{key} ({value})" for key, value in top.items()]

['b (2)', 'a (1)']

In [107]:
top.items()

In [23]:
xes.df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6660 entries, 0 to 1
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   org:group              6660 non-null   object             
 1   resource country       6660 non-null   object             
 2   organization country   6660 non-null   object             
 3   org:resource           6660 non-null   object             
 4   organization involved  6660 non-null   object             
 5   org:role               4582 non-null   object             
 6   concept:name           6660 non-null   object             
 7   impact                 6660 non-null   object             
 8   product                6660 non-null   object             
 9   lifecycle:transition   6660 non-null   object             
 10  time:timestamp         6660 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(10)
memory usage: 624.4+ KB


In [25]:
xes.df["org:group"].dtype.name

'object'

In [27]:
{ "a": 1 } | { "b": 2 }

{'a': 1, 'b': 2}